In [148]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo

In [157]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/nadiarichards/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


# NASA Mars News

In [168]:
mars_url = 'https://mars.nasa.gov/news/'
browser.visit(mars_url)
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [169]:
news_html = browser.html
news_soup = soup(news_html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [170]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8875/testing-proves-its-worth-with-successful-mars-parachute-deployment/" target="_self">Testing Proves Its Worth With Successful Mars Parachute Deployment</a></div>

In [171]:
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

'Testing Proves Its Worth With Successful Mars Parachute Deployment'

In [172]:
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'The giant canopy that helped land Perseverance on Mars was tested here on Earth at NASA’s Wallops Flight Facility in Virginia.'

# NASA Jet Propulsion Laboratory Images

In [173]:
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_image)
img_html = browser.html
img_soup = soup(img_html, 'html.parser')

In [174]:
featured_image_url = img_soup.find("img", class_="BaseImage object-contain")["data-src"]
featured_image_url

'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24414.2e16d0ba.fill-400x400-c50.jpg'

# Mars Facts Table

In [175]:
table_url="https://space-facts.com/mars/"

In [176]:
tables = pd.read_html(table_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [177]:
df=tables[1]
mars_df=df[["Mars - Earth Comparison", "Mars"]]
mars_df.columns = ['Description','Value']
mars_df.set_index('Description',inplace=True)
mars_df

,Value
Description,
Diameter:,"6,779 km"
Mass:,6.39 × 10^23 kg
Moons:,2
Distance from Sun:,"227,943,824 km"
Length of Year:,687 Earth days
Temperature:,-87 to -5 °C


In [178]:
html_table=mars_df.to_html()
html_table 

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n  </tbody>\n</table>'

# Mars Hemispheres

In [179]:
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)

In [180]:
hemisphere_image_urls = []
links = browser.find_by_css("a.product-item h3")
for i in range(len(links)):
    hemisphere = {}
    browser.find_by_css("a.product-item h3")[i].click()
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    hemisphere['title'] = browser.find_by_css("h2.title").text
    hemisphere_image_urls.append(hemisphere)
    browser.back()

In [181]:
hemisphere_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [182]:
browser.quit()